<style>
table {
    border-collapse: collapse;
    
}

table, th, td {
    border: 1px solid black;
    background-color: #4CAF50;
    color: white;
}
</style>

<table align="left" bgcolor="#FFFFFF">
    <tr>
        <td>
        <img style="float: left; padding-right: 30px" src="https://ecology.wa.gov/UI/images/ecylogo-wide-color.svg" alt="WA State Department of Ecology logo." width="600" height="300" border="3">
        </td>
        <td><p><span style="font-size:36px;">Environmental Justice Equity Analysis Mockup</span> </p>
            <p>Assess Distribution of locations across selected Environmental Justice indicator and evaluate service equity. EJ Indicators include EPA EJ Screen's WA State Minority Population Percentile and Low Income Percentile and WA Dept of Health's Washington Tracking Networks Rollup Environmental Health Disparities indicator and Socioeconomic indicator.
        </td>
    </tr><tr><td colspan=2>Questions? email <a href="mailto:tpow461@ecy.wa.gov">teague.powell@ecy.wa.gov</a></td></tr></table>
<p><br>
    <p>
        <p>

<p>


In [1]:
import pandas as pd
import geopandas as gpd
import folium
import ipywidgets as widgets
import pylab as pl, branca.colormap as bcm
import matplotlib as mpl, matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from IPython.core.display import HTML
np.seterr(divide='ignore', invalid='ignore',);

In [2]:
eval=pd.read_csv('evalXL.csv')
results = gpd.read_file('results.geojson')

SeaRedlines=gpd.read_file('WASeattle1936.geojson')
SpoRedlines=gpd.read_file('WASpokane1938.geojson')
TacRedlines=gpd.read_file('WATacoma1937.geojson')
redlines=pd.concat([SeaRedlines,SpoRedlines,TacRedlines],ignore_index=True)

In [3]:
from ipywidgets import Layout
from ipywidgets import FileUpload

style = {'description_width': 'initial'}

w_upload = FileUpload(description='upload excel file')

w_indicator = widgets.Dropdown(options=[ ('Env Health Disparities','EHDRank'),('SocioEconomic Risk','SERank'),('Low Income Population','LWINRank'),('Minority Population','MINRank')],
                 description='Indicator:',
                 disabled=False,
                 style=style,
                 layout = Layout(width='50%', height='40px', display='flex'))

w_ID = widgets.Dropdown(options=['please upload file first'],
                 description='Name/Identifier:',
                 disabled=False,
                 style=style,
                 layout = Layout(width='50%', height='40px', display='flex'))
w_Lat = widgets.Dropdown(options=['please upload file first'],
                 description='Lattitude / Y:',
                 disabled=False,
                 style=style,
                 layout = Layout(width='50%', height='40px', display='flex'))
w_Long = widgets.Dropdown(options=['please upload file first'],
                 description='Longitude / X:',
                 disabled=False,
                 style=style,
                 layout = Layout(width='50%', height='40px', display='flex'))




In [4]:
colormap2=bcm.LinearColormap(
        ['#053061','#2166ac','#4393c3','#4393c3','#4393c3','#d6604d','#b2182b','#b2182b','#b2182b','#67001f','#67001f'],
        index=[1,2,3,4,5,6,7,8,9,10],vmax=10)

colormap=bcm.LinearColormap(
        ['#053061','#2166ac','#4393c3','#92c5de','#d1e5f0','#fddbc7','#f4a582','#d6604d','#b2182b','#67001f','#67001f'],
        index=[1,2,3,4,5,6,7,8,9,10],vmax=10)

def get_color(value):
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    if value  =='A':
        return '#6ded5f' # MISSING -> gray
    if value  =='B':
        return '#e6f024' # MISSING -> gray
    if value  =='C':
        return '#d99029' # MISSING -> gray  
    if value  =='D':
        return '#a84632' # MISSING -> gray
    else:
        return color_scale(value)

In [23]:
def update_map(upload,id,lat,long,limit):

    
    try:
        sheets = pd.read_excel(w_upload.data[0],header=0)
        w_Long.options=sheets.columns.values
        w_Lat.options=sheets.columns.values
        w_ID.options=sheets.columns.values
        evaluated = gpd.GeoDataFrame(sheets, geometry=gpd.points_from_xy(sheets[w_Long.value],sheets[w_Lat.value]))
        evaluated.crs=('EPSG:4269')
        evaluated=gpd.sjoin(evaluated,results,how='left')
        evaluated=evaluated[~evaluated['SERank'].isnull()]
        evaluated['latitude']=evaluated[w_Lat.value]
        evaluated['longitude']=evaluated[w_Long.value]
        evaluated['name']=evaluated[w_ID.value]
        results['target']=results[limit]
        select_visits=evaluated
        #select_facilities=background[background['StateDistrict'].isin(region)]

        layer_alpha=.1
        color_alpha=.1
        m = folium.Map(location=[47.527212,-120.5469224], zoom_start=8, tiles='cartodbpositron', width='80%', height='80%')


        colormap.caption = 'Decile Rank (Low-High)'
        colormap.add_to(m)

        RL = folium.FeatureGroup(name='Redlining', show=False)
        RLchoropleth = folium.GeoJson(redlines.to_json(),
                                      style_function=lambda x :{'fillColor':get_color(x['properties']['holc_grade']),'color':get_color(x['properties']['holc_grade']),'weight':.5,'fillOpacity':.5},
                                name='result4',tooltip=folium.features.GeoJsonTooltip(fields=['name','holc_grade','area_description_data'], 
                                                                                      aliases=['name','grade','desc']))
        RLchoropleth.add_to(RL)
        RL.add_to(m)


        Visits = folium.FeatureGroup(name='Visits', show=True)
        #NoVisits = folium.FeatureGroup(name='Facilities', show=True)
        EJM = folium.FeatureGroup(name=w_indicator.label, show=False)
        CodedVisits = folium.FeatureGroup(name='Coded Visits', show=False)
        #CodedFacilities = folium.FeatureGroup(name='Coded Facilities', show=False)    

        EJMchoropleth = folium.GeoJson(results.to_json(),
                                name='result4',
                                highlight_function=lambda x: {'color':'#000000', 'weight':2,'fillOpacity':.5},
                                style_function=lambda x :{'fillColor':colormap(x['properties']['target']),
                                                          'color':colormap(x['properties']['target']),'weight':.5,'fillOpacity':color_alpha},
                                tooltip=folium.features.GeoJsonTooltip(fields=[limit], aliases=[limit], style=('background-color:rgba(255, 255, 255, 0.7); color: black; font-family:courier new; font-size: 7px; padding: 0px;'),sticky=True))

        select_visits.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]],
                                                  highlight_function=lambda x: {"weight":3,'fillOpacity':0.95,'zoom':8},
                                                  radius=5, color='green',weight=1,show=True, fill_color='green',
                                                           popup=folium.map.Popup(''+str(row['name']),max_width=150))
                                                 .add_to(Visits), axis=1)

        select_visits.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]],
                                                  highlight_function=lambda x: {"weight":3,'fillOpacity':0.95,'zoom':8},
                                                  radius=6, color=colormap2(row[limit]),weight=1,fill_opacity=.8, fill_color=colormap(row[limit]),
                                                           popup=folium.map.Popup(''+str(row['name']),max_width=200))
                                                 .add_to(CodedVisits), axis=1)


        Visits.add_to(m)

        CodedVisits.add_to(m)

        m.keep_in_front(Visits)
        m.keep_in_front(CodedVisits)

        EJMchoropleth.add_to(EJM)
        EJM.add_to(m)

        folium.TileLayer('cartodbpositron').add_to(m)
        folium.LayerControl().add_to(m)

        title = title_html = '''
         <head><style> html { overflow-y: hidden; } </style></head>
         <h3 align="center" style="font-size:20px"><b>Planning Facilities and Visits Across Vulnerable Communities ('''+w_indicator.label+''')</b></h3>
         ''' 

        title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title)   
        m.get_root().html.add_child(folium.Element(title_html))

        display(m)


        fig = plt.figure(figsize=(9,6),constrained_layout=True)
        gs1 = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)


        plot2 = fig.add_subplot(gs1[0,0])
        N, bins, patches2 = pl.hist(evaluated[limit], bins=np.arange(1,12)-.5)
        for i in range(len(patches2)):
            patches2[i].set_facecolor(colormap(i))
        plt.grid(b=False, which='major', color='#999999', linestyle='-', alpha=.2)
        plt.xticks(np.arange(1, 11, 1.0))
        plt.xlabel(limit, fontsize=16)  
        plt.ylabel("Visits", fontsize=16)
        plt.title("TR Visits by "+limit, fontsize=16)
        plt.show()




        #priority=background2.merge(eval2[['FacilityName','StaffReminderDate']], on='FacilityName', how='outer')
        display(select_visits[['name','EHDRank','SERank','MINRank','LWINRank']].reset_index(drop=True).sort_values([limit], ascending=False).style.format({'EHDRank':"{:.0f}",'SERank':"{:.0f}",'MINRank':"{:.0f}",'LWINRank':"{:.0f}"}).background_gradient(cmap='coolwarm',subset=[limit]))


#    except:
    except Exception as e: print(e)

In [24]:
widgets.interactive(update_map, upload=w_upload, limit=w_indicator, id=w_ID, lat=w_Lat, long=w_Long, )

interactive(children=(FileUpload(value={'evalXL.xlsx': {'metadata': {'name': 'evalXL.xlsx', 'type': 'applicati…